# MACHINE LEARNING NO SUPERVISADO
## PCA - Caso de Estudio

Ahora, apliquemos el PCA para reducir la dimensionalidad del conjunto de datos MNIST. Utilizaremos la versión comprimida (1,797 imágenes de 8 × 8) proporcionada por scikit-learn, pero ninguna de nuestras consideraciones se verá afectada por esta elección. Comencemos cargando y normalizando el conjunto de datos:


In [ ]:
import numpy as np
from sklearn.datasets import load_digits

# For reproducibility
np.random.seed(1000)
# Load the dataset
digits = load_digits()
X = digits['data'] / np.max(digits['data'])

A partir de la discusión teórica, sabemos que la magnitud de los eigenvalores de la matriz de covarianza es proporcional a la importancia relativa (es decir, la varianza explicada y, por lo tanto, el contenido informativo) del componente principal correspondiente. Por lo tanto, si se ordenan en orden descendente, es posible calcular las siguientes diferencias:

$sdfld,aslda$

Como la importancia tiende a disminuir cuando el número de componentes \( k \) se acerca a \( n \), podemos seleccionar el \( k \) óptimo eligiendo la primera mayor diferencia, la cual indica una disminución sustancial en la cantidad de varianza explicada por todos los componentes siguientes. Para comprender mejor este mecanismo, calculemos los eigenvalores y sus diferencias (como la matriz de covarianza, \( C \), es semidefinida positiva, estamos seguros de que \( \lambda_i \geq 0 \) para todo \( i \) en el rango de 1 a \( n \)). Los resultados se pueden ver a continuación

(insertar imagen)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Compute the eigenvalues of C
C = np.cov(X.T)
l, v = np.linalg.eig(C)
l = np.sort(l)[::-1]
d = l[:l.shape[0] - 1] - l[1:]

# Show the differences
sns.set()
fig, ax = plt.subplots(figsize=(22, 12))
ax.bar(np.arange(2, len(d) + 2, 1), d)
ax.set_xticks(np.arange(2, len(d) + 2, 1))
ax.set_xlabel('Component', fontsize=16)
ax.set_ylabel('Eigenvalue difference', fontsize=16)
plt.show()

Como se puede observar, las diferencias para los primeros componentes principales son muy grandes, alcanzando un máximo en correspondencia con el cuarto componente principal (\( \lambda_4 - \lambda_3 \)); sin embargo, la siguiente diferencia sigue siendo muy alta, mientras que hay una caída abrupta correspondiente a \( \lambda_6 \). En este punto, la tendencia es casi estable (excepto por algunas oscilaciones residuales) hasta \( \lambda_{11} \), y luego comienza a disminuir muy rápidamente, tendiendo hacia cero. Dado que aún deseamos tener imágenes cuadradas, vamos a elegir \( k = 16 \) (lo que equivale a dividir cada lado por cuatro). En otro escenario, podrías elegir, por ejemplo, \( k = 15 \), o incluso \( k = 8 \); sin embargo, para tener una mejor comprensión del error inducido por la reducción de dimensionalidad, será útil analizar también la varianza explicada. Por lo tanto, comencemos realizando el Análisis de Componentes Principales (PCA).